# Intro to Jupyter Notebook

The information included here follows these webpages: https://realpython.com/jupyter-notebook-introduction/, https://medium.com/ibm-data-science-experience/back-to-basics-jupyter-notebooks-dfcdc19c54bc

## About this Notebook

### Requirements
- Direct access to the input data
- Jupyter notebook installed on that platform
- Python packages:
    - Matplotlib
    - mpl_toolkits
    - NetCDF4
    - Numpy
    
    

### Modes
- The cells are either Python 3 code (next cell), or Markdown (this cell). 
- Modes can be in either of two modes:
    - `EDIT`, which appears green
    - `COMMAND`, which appears blue
- To enter enter `EDIT` mode, hit `Enter` or double click
- To enter `COMMAND` mode, hit `Esc` or click anywhere outside the box

### Navigation
The menus at the top of the page allow you to control the cells, run them, and manage the file. 

### Keyboard commands
Many of the above navigation menus have keyboard shortcuts:

- Command Mode: ESC
- Edit Mode: ENTER
- Run selected cell: CTRL+ENTER
- Run cell and insert below: ALT+ENTER
- Run cell and select below: SHIFT+ENTER

From COMMAND mode:
- Insert cell above: A
- Insert cell below: B
- Cut selected cells: X
- Copy selected cells: C
- Delete selected cell : d+d

## Run the notebook

To run the whole notebook, use the menu: `Kernel > Restart & Run All`, or click the "Fast forward" button on the top menu bar.

To run (or rerun) a single cell, click on the cell and press `Shift, Enter`, or click the "Run" button on the top menu bar.
    

In [ ]:
import os

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import shiftgrid
import numpy as np
import pygrib
from netCDF4 import Dataset

# Set up the input files on Hera

Abstractions to change date and location were made, but assuming the final directory structure would remain the same.

In [ ]:
# Set files
bkg_date = '2019080112'
bkg_loc = f'/path/to/dir/data/{bkg_date}'
bkg_dynam_nc = os.path.join(bkg_loc, 'RESTART', 'fv_core.res.tile1.nc')
bkg_tracer_nc = os.path.join(bkg_loc, 'RESTART', 'fv_tracer.res.tile1.nc')
bkg_sfc_nc = os.path.join(bkg_loc, 'RESTART', 'sfc_data.nc')
bkg_map_nc = os.path.join(bkg_loc, 'grid_spec.nc')

expname = 'prepbufr'
anl_date = '2019080300'
anl_loc = f'/path/to/dir/{anl_date}/{anl_date}.{expname}.hrrr'
anl_dynam_nc = os.path.join(anl_loc, 'fv3_dynvars')
anl_tracer_nc = os.path.join(anl_loc, 'fv3_tracer')
anl_sfc_nc = os.path.join(anl_loc, 'fv3_sfcdata')
anl_map_nc = os.path.join(anl_loc, 'fv3_grid_spec')

# Load files into NetCDF Datasets
bkg_dynam = Dataset(bkg_dynam_nc, 'r')
bkg_tracer = Dataset(bkg_tracer_nc, 'r')
bkg_sfc = Dataset(bkg_sfc_nc, 'r')
bkg_map = Dataset(bkg_map_nc, 'r')

anl_dynam = Dataset(anl_dynam_nc, 'r')
anl_tracer = Dataset(anl_tracer_nc, 'r')
anl_sfc = Dataset(anl_sfc_nc, 'r')
anl_map = Dataset(anl_map_nc, 'r')

In [ ]:
# Load latitude/longitude from background grid spec file
grid_lon = bkg_map['grid_lon'][::]
grid_lat = bkg_map['grid_lat'][::]

In [ ]:
# var_loc is a dictionary that describes which Python variables contain the desired atmospheric variables.
# For example, the sphum variable is found only in the tracer files, while u, v, and T are found in the dynam files.
var_loc = {
    'u': {
        'anl': anl_dynam,
        'bkg': bkg_dynam,
    },
    'v':  {
        'anl': anl_dynam,
        'bkg': bkg_dynam,
    },
    'T':  {
        'anl': anl_dynam,
        'bkg': bkg_dynam,
    },
    'sphum':  {
        'anl': anl_tracer,
        'bkg': bkg_tracer,
    },
}

In [ ]:
def plot_data(data, lat, lon, title):
    
    '''
    Input parameters:
    
        data: 2D Numpy array to be plotted
        lat: 2D Numpy array of latitude
        lon: 2D Numpy array of longitude
        var: String describing the variable being plotted.
        
    Draws a Basemap representation with the contoured data overlayed, with a colorbar.
        
    '''
    
    def trim_grid():
        '''
        The u, v, and H data are all on grids either one column, or one row smaller than lat/lon. 
        Return the smaller lat, lon grids, given the shape of the data to be plotted.
        '''
        y, x = np.shape(data)
        return lat[:y, :x], lon[:y, :x]
    
    m = Basemap(projection='mill', 
                llcrnrlon=lon.min()-5,
                urcrnrlon=lon.max()+5,
                llcrnrlat=lat.min()-5,
                urcrnrlat=lat.max()+5,
                resolution='c',
               )

    lat_trim, lon_trim = trim_grid()
    plt.figure(figsize=(12,12))
    x, y = m(lon_trim, lat_trim)

    cs = m.contourf(x, y, data)
    m.drawcoastlines();
    m.drawmapboundary();
    m.drawparallels(np.arange(-90.,120.,5),labels=[1,0,0,0]);
    m.drawmeridians(np.arange(-180.,180.,5),labels=[0,0,0,1]);
    plt.colorbar(cs,orientation='vertical', shrink=0.5);
    plt.title(f"{title}")

# Plot the variables

Loop through each of the variables contained in the keys of `var_loc`, get the data from the appropriate Python variable, and plot the results.

In [ ]:
for var in var_loc.keys():
    lev = 0   # Change this variable to look at different vertical levels.
    anl, bkg = [var_loc[var][stage][var][::] for stage in ['anl', 'bkg']]
    title = f'Analysis Increment for {var}'
    plot_data(np.squeeze(anl)[lev, :, :]-np.squeeze(bkg)[lev, :, :], grid_lat, grid_lon, title)


# Explore the data

In [ ]:
# Print just the variable names:
print('Dynamics variables include')
for var in anl_dynam.variables:
    print(var)

In [ ]:
# Print more information about the variables
print('Dynamics file contains: ')
for var, info in anl_dynam.variables.items():
    print(var, '------------\n', info)
    

In [ ]:
# Access the data in a variable
u = anl_dynam['u'][::]
u


In [ ]:
# What is the size of the data?
np.shape(u)